# Converter for the Financial Filings at the Sec

In [53]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2012q1.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t")
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2012q1"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

23142411
11393480
267012
980622
24168


In [54]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+-------------------------------------------------------+--------------+---------------------------------------+----------+--------+-------+-------------------+------------+
|    | adsh                 | tag                                                   | version      | coreg                                 |    ddate |   qtrs | uom   |             value |   footnote |
|----+----------------------+-------------------------------------------------------+--------------+---------------------------------------+----------+--------+-------+-------------------+------------|
|  0 | 0001518519-12-000030 | AcceleratedShareRepurchaseProgramAdjustment           | us-gaap/2011 | nan                                   | 20101231 |      4 | USD   |      -5.8355e+07  |        nan |
|  1 | 0001518519-12-000030 | AcceleratedShareRepurchaseProgramAdjustment           | us-gaap/2011 | nan                                   | 20111231 |      4 | USD   |      -1.414e+07   |    

In [55]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                   | version      | plabel                                                                                                                                                  |   negating |
|----+----------------------+----------+--------+--------+---------+---------+-------------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+------------|
|  0 | 0001052918-12-000148 |        7 |     23 |

In [56]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+---------------------------------------------------------------+-------+-------------+----------+-------------------------+------------+----------------------------------------+----------------------------------------+-----------------+-------------+----------+-------------------------+------------+------------------------------------------+-------------------------------------+--------------+-----------+-------------+-------------------------------------------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+----------------------------+---------+---------+
|    | adsh                 |     cik | name                                                          |   sic | countryba   | stprba   | cityba                  | zipba      | bas1                                   | bas2                                   | baph            | countr

In [57]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+-----------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                                          

In [58]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [59]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                               0001518519-12-000030
tag         AcceleratedShareRepurchaseProgramAdjustment
version                                    us-gaap/2011
coreg                                               NaN
ddate                                          20101231
qtrs                                                  4
uom                                                 USD
value                                       -58355000.0
footnote                                            NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [60]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

# Create a single SymbolFinancialsDto

In [61]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    sfDto.symbol = val["symbol"].to_string(index = False).upper()
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False)
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = result.replace("'",'"')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(str(result))
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 195 Dtos size: 124 Bs: 72 Cf: 52 Ic: 0 Json size: 40kb.
file 1 of 7418 exportFiles/2012q1/0000040194-12-000010.json stored in 4713.108ms.
FilterDfNum size: 154 Dtos size: 107 Bs: 62 Cf: 45 Ic: 0 Json size: 29kb.
file 2 of 7418 exportFiles/2012q1/0000061004-12-000012.json stored in 3819.143ms.
FilterDfNum size: 536 Dtos size: 148 Bs: 79 Cf: 69 Ic: 0 Json size: 39kb.
file 3 of 7418 exportFiles/2012q1/0000082166-12-000012.json stored in 13253.061ms.
FilterDfNum size: 205 Dtos size: 136 Bs: 78 Cf: 58 Ic: 0 Json size: 34kb.
file 4 of 7418 exportFiles/2012q1/0000107687-12-000017.json stored in 5151.069ms.
FilterDfNum size: 321 Dtos size: 174 Bs: 72 Cf: 102 Ic: 0 Json size: 43kb.
file 5 of 7418 exportFiles/2012q1/0000315849-12-000007.json stored in 8280.552ms.
FilterDfNum size: 277 Dtos size: 180 Bs: 74 Cf: 106 Ic: 0 Json size: 45kb.
file 6 of 7418 exportFiles/2012q1/0000706863-12-000007.json stored in 6805.566ms.
FilterDfNum size: 97 Dtos size: 58 Bs: 20 Cf: 38 Ic: 0 Json s

FilterDfNum size: 375 Dtos size: 191 Bs: 78 Cf: 113 Ic: 0 Json size: 47kb.
file 54 of 7418 exportFiles/2012q1/0000950159-12-000191.json stored in 8584.449ms.
FilterDfNum size: 87 Dtos size: 52 Bs: 31 Cf: 21 Ic: 0 Json size: 14kb.
file 55 of 7418 exportFiles/2012q1/0001000096-12-000066.json stored in 2095.682ms.
FilterDfNum size: 171 Dtos size: 103 Bs: 45 Cf: 58 Ic: 0 Json size: 27kb.
file 56 of 7418 exportFiles/2012q1/0001002014-12-000180.json stored in 3975.267ms.
FilterDfNum size: 325 Dtos size: 185 Bs: 80 Cf: 105 Ic: 0 Json size: 47kb.
file 57 of 7418 exportFiles/2012q1/0001002105-12-000040.json stored in 8202.377ms.
FilterDfNum size: 178 Dtos size: 119 Bs: 63 Cf: 56 Ic: 0 Json size: 29kb.
file 58 of 7418 exportFiles/2012q1/0001002334-12-000048.json stored in 4322.886ms.
FilterDfNum size: 217 Dtos size: 142 Bs: 59 Cf: 83 Ic: 0 Json size: 39kb.
file 59 of 7418 exportFiles/2012q1/0001003297-12-000141.json stored in 5321.521ms.
FilterDfNum size: 112 Dtos size: 71 Bs: 32 Cf: 39 Ic: 0 Js

FilterDfNum size: 198 Dtos size: 145 Bs: 83 Cf: 62 Ic: 0 Json size: 37kb.
file 107 of 7418 exportFiles/2012q1/0001036262-12-000010.json stored in 4657.958ms.
FilterDfNum size: 247 Dtos size: 134 Bs: 68 Cf: 66 Ic: 0 Json size: 34kb.
file 108 of 7418 exportFiles/2012q1/0001040799-12-000002.json stored in 5734.822ms.
FilterDfNum size: 162 Dtos size: 86 Bs: 44 Cf: 42 Ic: 0 Json size: 26kb.
file 109 of 7418 exportFiles/2012q1/0001043951-12-000010.json stored in 3786.76ms.
FilterDfNum size: 242 Dtos size: 138 Bs: 50 Cf: 88 Ic: 0 Json size: 36kb.
file 110 of 7418 exportFiles/2012q1/0001047469-12-003586.json stored in 5687.48ms.
FilterDfNum size: 246 Dtos size: 136 Bs: 66 Cf: 70 Ic: 0 Json size: 35kb.
file 111 of 7418 exportFiles/2012q1/0001047469-12-003603.json stored in 5725.502ms.
FilterDfNum size: 180 Dtos size: 126 Bs: 54 Cf: 72 Ic: 0 Json size: 33kb.
file 112 of 7418 exportFiles/2012q1/0001047469-12-003650.json stored in 4237.088ms.
FilterDfNum size: 296 Dtos size: 210 Bs: 97 Cf: 113 Ic:

FilterDfNum size: 163 Dtos size: 114 Bs: 71 Cf: 43 Ic: 0 Json size: 28kb.
file 160 of 7418 exportFiles/2012q1/0001096906-12-000765.json stored in 3782.09ms.
FilterDfNum size: 157 Dtos size: 82 Bs: 42 Cf: 40 Ic: 0 Json size: 21kb.
file 161 of 7418 exportFiles/2012q1/0001096906-12-000768.json stored in 3470.567ms.
FilterDfNum size: 292 Dtos size: 189 Bs: 84 Cf: 105 Ic: 0 Json size: 44kb.
file 162 of 7418 exportFiles/2012q1/0001096906-12-000774.json stored in 6547.945ms.
FilterDfNum size: 58 Dtos size: 36 Bs: 26 Cf: 10 Ic: 0 Json size: 9kb.
file 163 of 7418 exportFiles/2012q1/0001096906-12-000776.json stored in 1429.14ms.
FilterDfNum size: 96 Dtos size: 41 Bs: 41 Cf: 0 Ic: 0 Json size: 10kb.
file 164 of 7418 exportFiles/2012q1/0001102624-12-000251.json stored in 2250.288ms.
FilterDfNum size: 97 Dtos size: 46 Bs: 46 Cf: 0 Ic: 0 Json size: 11kb.
file 165 of 7418 exportFiles/2012q1/0001102624-12-000252.json stored in 2291.79ms.
FilterDfNum size: 199 Dtos size: 122 Bs: 64 Cf: 58 Ic: 0 Json si

FilterDfNum size: 251 Dtos size: 173 Bs: 74 Cf: 99 Ic: 0 Json size: 44kb.
file 213 of 7418 exportFiles/2012q1/0001142417-12-000016.json stored in 5486.818ms.
FilterDfNum size: 276 Dtos size: 142 Bs: 82 Cf: 60 Ic: 0 Json size: 38kb.
file 214 of 7418 exportFiles/2012q1/0001144204-12-018379.json stored in 6105.016ms.
FilterDfNum size: 114 Dtos size: 55 Bs: 22 Cf: 33 Ic: 0 Json size: 14kb.
file 215 of 7418 exportFiles/2012q1/0001144204-12-018381.json stored in 2568.146ms.
FilterDfNum size: 65 Dtos size: 44 Bs: 32 Cf: 12 Ic: 0 Json size: 12kb.
file 216 of 7418 exportFiles/2012q1/0001144204-12-018386.json stored in 1532.225ms.
FilterDfNum size: 196 Dtos size: 146 Bs: 70 Cf: 76 Ic: 0 Json size: 39kb.
file 217 of 7418 exportFiles/2012q1/0001144204-12-018388.json stored in 4338.605ms.
FilterDfNum size: 102 Dtos size: 46 Bs: 20 Cf: 26 Ic: 0 Json size: 13kb.
file 218 of 7418 exportFiles/2012q1/0001144204-12-018396.json stored in 2320.015ms.
FilterDfNum size: 153 Dtos size: 106 Bs: 59 Cf: 47 Ic: 0

...

FilterDfNum size: 170 Dtos size: 96 Bs: 58 Cf: 38 Ic: 0 Json size: 26kb.
file 7188 of 7418 exportFiles/2012q1/0001171520-12-000021.json stored in 3893.159ms.
FilterDfNum size: 213 Dtos size: 145 Bs: 79 Cf: 66 Ic: 0 Json size: 38kb.
file 7189 of 7418 exportFiles/2012q1/0001171520-12-000024.json stored in 4822.411ms.
FilterDfNum size: 97 Dtos size: 58 Bs: 30 Cf: 28 Ic: 0 Json size: 17kb.
file 7190 of 7418 exportFiles/2012q1/0001171520-12-000025.json stored in 2297.08ms.
FilterDfNum size: 180 Dtos size: 123 Bs: 78 Cf: 45 Ic: 0 Json size: 32kb.
file 7191 of 7418 exportFiles/2012q1/0001171843-12-000110.json stored in 4106.117ms.
FilterDfNum size: 82 Dtos size: 50 Bs: 33 Cf: 17 Ic: 0 Json size: 14kb.
file 7192 of 7418 exportFiles/2012q1/0001185185-12-000067.json stored in 1943.158ms.
FilterDfNum size: 213 Dtos size: 133 Bs: 51 Cf: 82 Ic: 0 Json size: 33kb.
file 7193 of 7418 exportFiles/2012q1/0001193125-12-011060.json stored in 4784.961ms.
FilterDfNum size: 136 Dtos size: 95 Bs: 52 Cf: 43 Ic

FilterDfNum size: 320 Dtos size: 197 Bs: 99 Cf: 98 Ic: 0 Json size: 53kb.
file 7240 of 7418 exportFiles/2012q1/0001477932-12-000059.json stored in 7191.2ms.
FilterDfNum size: 147 Dtos size: 90 Bs: 54 Cf: 36 Ic: 0 Json size: 22kb.
file 7241 of 7418 exportFiles/2012q1/0001477932-12-000060.json stored in 3315.128ms.
FilterDfNum size: 78 Dtos size: 50 Bs: 32 Cf: 18 Ic: 0 Json size: 13kb.
file 7242 of 7418 exportFiles/2012q1/0001504167-12-000002.json stored in 1853.976ms.
FilterDfNum size: 268 Dtos size: 168 Bs: 80 Cf: 88 Ic: 0 Json size: 42kb.
file 7243 of 7418 exportFiles/2012q1/0001513162-12-000052.json stored in 5941.686ms.
FilterDfNum size: 45 Dtos size: 36 Bs: 22 Cf: 14 Ic: 0 Json size: 9kb.
file 7244 of 7418 exportFiles/2012q1/0001515250-12-000002.json stored in 1120.321ms.
FilterDfNum size: 49 Dtos size: 36 Bs: 22 Cf: 14 Ic: 0 Json size: 9kb.
file 7245 of 7418 exportFiles/2012q1/0001515250-12-000003.json stored in 1284.227ms.
FilterDfNum size: 49 Dtos size: 36 Bs: 22 Cf: 14 Ic: 0 Js

FilterDfNum size: 102 Dtos size: 55 Bs: 28 Cf: 27 Ic: 0 Json size: 15kb.
file 7292 of 7418 exportFiles/2012q1/0001524777-12-000006.json stored in 2414.429ms.
FilterDfNum size: 79 Dtos size: 53 Bs: 28 Cf: 25 Ic: 0 Json size: 14kb.
file 7293 of 7418 exportFiles/2012q1/0001524777-12-000008.json stored in 1888.183ms.
FilterDfNum size: 281 Dtos size: 112 Bs: 60 Cf: 52 Ic: 0 Json size: 29kb.
file 7294 of 7418 exportFiles/2012q1/0000723125-12-000007.json stored in 6209.753ms.
FilterDfNum size: 174 Dtos size: 101 Bs: 65 Cf: 36 Ic: 0 Json size: 27kb.
file 7295 of 7418 exportFiles/2012q1/0001072613-12-000010.json stored in 3972.176ms.
FilterDfNum size: 160 Dtos size: 101 Bs: 38 Cf: 63 Ic: 0 Json size: 27kb.
file 7296 of 7418 exportFiles/2012q1/0001078782-12-000044.json stored in 3639.29ms.
FilterDfNum size: 72 Dtos size: 32 Bs: 18 Cf: 14 Ic: 0 Json size: 8kb.
file 7297 of 7418 exportFiles/2012q1/0001091818-12-000008.json stored in 1709.746ms.
FilterDfNum size: 177 Dtos size: 95 Bs: 56 Cf: 39 Ic:

FilterDfNum size: 75 Dtos size: 41 Bs: 34 Cf: 7 Ic: 0 Json size: 12kb.
file 7344 of 7418 exportFiles/2012q1/0001471242-12-000022.json stored in 1873.521ms.
FilterDfNum size: 65 Dtos size: 40 Bs: 16 Cf: 24 Ic: 0 Json size: 10kb.
file 7345 of 7418 exportFiles/2012q1/0001501112-12-000002.json stored in 1628.637ms.
FilterDfNum size: 53 Dtos size: 42 Bs: 34 Cf: 8 Ic: 0 Json size: 11kb.
file 7346 of 7418 exportFiles/2012q1/0001529293-12-000002.json stored in 1366.172ms.
FilterDfNum size: 374 Dtos size: 137 Bs: 72 Cf: 65 Ic: 0 Json size: 34kb.
file 7347 of 7418 exportFiles/2012q1/0000068270-12-000008.json stored in 8579.779ms.
FilterDfNum size: 279 Dtos size: 150 Bs: 82 Cf: 68 Ic: 0 Json size: 38kb.
file 7348 of 7418 exportFiles/2012q1/0000950123-12-000364.json stored in 6572.066ms.
FilterDfNum size: 158 Dtos size: 97 Bs: 64 Cf: 33 Ic: 0 Json size: 24kb.
file 7349 of 7418 exportFiles/2012q1/0000950123-12-000426.json stored in 3757.084ms.
FilterDfNum size: 70 Dtos size: 46 Bs: 34 Cf: 12 Ic: 0 

FilterDfNum size: 170 Dtos size: 124 Bs: 64 Cf: 60 Ic: 0 Json size: 32kb.
file 7396 of 7418 exportFiles/2012q1/0000809933-12-000004.json stored in 3829.935ms.
FilterDfNum size: 175 Dtos size: 107 Bs: 64 Cf: 43 Ic: 0 Json size: 28kb.
file 7397 of 7418 exportFiles/2012q1/0000950123-12-000093.json stored in 3933.689ms.
FilterDfNum size: 214 Dtos size: 115 Bs: 64 Cf: 51 Ic: 0 Json size: 31kb.
file 7398 of 7418 exportFiles/2012q1/0001104659-12-000340.json stored in 4879.133ms.
FilterDfNum size: 97 Dtos size: 43 Bs: 20 Cf: 23 Ic: 0 Json size: 12kb.
file 7399 of 7418 exportFiles/2012q1/0001185185-12-000009.json stored in 2229.82ms.
FilterDfNum size: 102 Dtos size: 65 Bs: 39 Cf: 26 Ic: 0 Json size: 17kb.
file 7400 of 7418 exportFiles/2012q1/0001185185-12-000012.json stored in 2344.663ms.
FilterDfNum size: 265 Dtos size: 131 Bs: 85 Cf: 46 Ic: 0 Json size: 35kb.
file 7401 of 7418 exportFiles/2012q1/0001193125-12-001629.json stored in 5824.644ms.
FilterDfNum size: 189 Dtos size: 113 Bs: 61 Cf: 52